In [ ]:
# Define model
# This formulation is more deliberate about creating constraint terms and fixing variables at 0, in an attempt to make the model smaller and faster
# It also has switches to include/exclude constraints and alternative objective functions

def DefineModel(Model):
    print('Defining model...')
    Model.Allocation = pyo.Var(Model.Candidate, Model.GridWords, within = pyo.Binary, initialize = 0)   # Allocate candidate words to the grid

    def rule_PosOnce(Model, g):   # Allocate exactly one candidate to each grid word position
        ColumnTotal = 0
        TermsAdded = 0
        for c in Model.Candidate:
            if pyo.value(Model.Length[c]) == pyo.value(Model.GridLengths[g]):   # Candidate word length must match grid word length
                ColumnTotal += Model.Allocation[c, g]
                TermsAdded += 1
        if TermsAdded == 0:
            return pyo.Constraint.Skip
        else:
            return ColumnTotal == 1
    Model.EachPositionOnce = pyo.Constraint(Model.GridWords, rule = rule_PosOnce)

    if SingleWordSquare:
        def rule_Symmetry(Model, g):   # Ensure that across words = down words, for creating single word squares
            NumWords = len(Model.GridWords)   # Always an even number in a square grid
            if g <= NumWords / 2 - 1:
                return sum(Model.Allocation[c, g] for c in Model.Candidate) == sum(Model.Allocation[c, g + NumWords / 2] for c in Model.Candidate)
            else:
                return pyo.Constraint.Skip
        Model.Symmetry = pyo.Constraint(Model.GridWords, rule = rule_Symmetry)
    
    def rule_WordOnce(Model, c):
        RowTotal = 0
        TermsAdded = 0
        if SingleWordSquare:
            rhs = 2   # Each allocated word is used twice (across and down)
        else:
            rhs = 1   # Each allocated word must be unique
        for g in Model.GridWords:
            if pyo.value(Model.Length[c]) == pyo.value(Model.GridLengths[g]):   # Candidate word length must match grid word length
                RowTotal += Model.Allocation[c, g]
                TermsAdded += 1
        if TermsAdded == 0:
            return pyo.Constraint.Skip
        else:
            return RowTotal <= rhs
    Model.EachWordOnce = pyo.Constraint(Model.Candidate, rule = rule_WordOnce)

    # Fix some variables. Replaces the rule_Fit constraint. This constraint substantially reduces solve time
    for g in Model.GridWords:   # Reduce variables by fixing allocation variables to zero if candidate word length doesn't match grid word length
        for c in Model.Candidate:    
            if pyo.value(Model.Length[c]) != pyo.value(Model.GridLengths[g]):
                Model.Allocation[c, g].fix(0)

    def rule_Intersection(Model, g1, g2, w, h):   # The intersection of grid words must have the same letter
                                                  # The if statements within the sums remove terms that are fixed at zero, so no change in reported model size
        if pyo.value(Model.AcrossRef[h, w]) != 0 \
                and pyo.value(Model.DownRef[h, w]) != 0 \
                and g1 == pyo.value(Model.AcrossRef[h, w]) - 1 \
                and g2 == pyo.value(Model.DownRef[h, w]) - 1:
            return sum(Model.Allocation[c, g1] * Model.Word[c, pyo.value(Model.AcrossPos[h, w]) - 1] for c in Model.Candidate \
                    if pyo.value(Model.Length[c]) == pyo.value(Model.GridLengths[g1])) == \
                    sum(Model.Allocation[c, g2] * Model.Word[c, pyo.value(Model.DownPos[h, w]) - 1] for c in Model.Candidate \
                    if pyo.value(Model.Length[c]) == pyo.value(Model.GridLengths[g2]))
        else:
            return pyo.Constraint.Skip
    Model.Crossover = pyo.Constraint(Model.GridWords, Model.GridWords, Model.GridWidth, Model.GridHeight, rule = rule_Intersection)

    CandidateUsed = []   # Use and fix randomly selected words at most once each
    if len(Model.FixGridWord) == 0:
        print('No initial words')
    else:   # Randomly select words to fill specified grid word positions
        print('Initial words:')
        for f in range(0, len(Model.FixGridWord)):
            CandidateList = []
            for c in Model.Candidate:
                if pyo.value(Model.Length[c]) == pyo.value(Model.GridLengths[pyo.value(Model.FixGridWord[f]) - 1]) and c not in CandidateUsed:
                    CandidateList.append(c)
            CandidateUsed = rnd.sample(CandidateList, 1)
            Model.Allocation[CandidateUsed, pyo.value(Model.FixGridWord[f]) - 1].fix(1)   # Fix variable for allocation of selected word
            CurrWord = ""
            for i in range(0, MaxWordLength):
                CurrWord += chr(pyo.value(Model.Word[CandidateUsed, i]))
            print(pyo.value(Model.FixGridWord[f]), ": ", CurrWord)   # Output selected word
    print()

    if ObjectiveChoice == 3:
        Model.Cross = pyo.Set(initialize = range(0, pyo.value(Model.NumIntersections)))
        Model.Same = pyo.Var(Model.Cross, within = pyo.Binary, initialize = 0)
        Model.Inter = pyo.ConstraintList()
        CurrZ = 0
        for h in Model.GridHeight:
            for w in Model.GridWidth:
                for g1 in Model.GridWords:
                    for g2 in Model.GridWords:
                        if pyo.value(Model.AcrossRef[h, w]) != 0 \
                                and pyo.value(Model.DownRef[h, w]) != 0 \
                                and g1 == pyo.value(Model.AcrossRef[h, w]) - 1 \
                                and g2 == pyo.value(Model.DownRef[h, w]) - 1:
                            Model.Inter.add(sum(Model.Allocation[c, g1] * Model.Word[c, pyo.value(Model.AcrossPos[h, w]) - 1] \
                                    for c in Model.Candidate if pyo.value(Model.Length[c]) == pyo.value(Model.GridLengths[g1])) \
                                    - sum(Model.Allocation[c, g2] * Model.Word[c, pyo.value(Model.DownPos[h, w]) - 1] for c in Model.Candidate \
                                    if pyo.value(Model.Length[c]) == pyo.value(Model.GridLengths[g2])) >= 1 - 26 * Model.Same[CurrZ])
                            CurrZ += 1

    def rule_Obj(Model):   # Min or Max total allocated word frequency. The if statement removes terms that are fixed at zero, so no change in reported model size
                           # Weighting by Model.Rank[c] rather than Model.Frequency[c] is sometimes faster. Though, to be consistent, also need to change the objective's direction
        if ObjectiveChoice == 3:
            return sum(Model.Same[z] for z in Model.Cross)
        elif ObjectiveChoice == 2:
            return sum(Model.Allocation[c, g] for g in Model.GridWords for c in Model.Candidate if pyo.value(Model.Length[c]) == pyo.value(Model.GridLengths[g]))
        else:   # option 1
            return sum(sum(Model.Allocation[c, g] for g in Model.GridWords) * Model.Frequency[c] for c in Model.Candidate \
                    if pyo.value(Model.Length[c]) == pyo.value(Model.GridLengths[g]))            
    if Direction == 1:
        Model.Obj = pyo.Objective(rule = rule_Obj, sense = pyo.maximize)
    else:
        Model.Obj = pyo.Objective(rule = rule_Obj, sense = pyo.minimize)